# Установка и импорт библиотек

In [37]:
!pip install openpyxl
!pip install catboost
!pip install optuna
!pip install scikit-learn

In [39]:
# Импорт библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
import optuna

In [40]:
# Загрузка файла
# Чтение данных
df = pd.read_csv('data/synthetic.csv')

# Проверка
print(df.head())

   Unnamed: 0              NDVI,LST,BURNED_AREA,CLASS
0           0  0.506782,14584.272727,4.692308,no_fire
1           1        0.52215,14655.833333,5.0,no_fire
2           2               0.682284,14780.0,5.0,fire
3           3            0.120046,13298.5,3.5,no_fire
4           4            0.568734,14743.0,5.0,no_fire


In [41]:
# Подготовка данных
df = df['NDVI,LST,BURNED_AREA,CLASS'].str.split(',', expand=True)
df.columns = ['NDVI', 'LST', 'BURNED_AREA', 'CLASS']

# Преобразование типов данных
df['NDVI'] = df['NDVI'].astype(float)
df['LST'] = df['LST'].astype(float)
df['BURNED_AREA'] = df['BURNED_AREA'].astype(float)
le = LabelEncoder()
df['CLASS'] = le.fit_transform(df['CLASS'])  # 1 - fire, 0 - no_fire

# Разделение данных
X = df[['NDVI', 'LST', 'BURNED_AREA']]
y = df['CLASS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [42]:
# Функция для оптимизации гиперпараметров с Optuna
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'auto_class_weights': 'Balanced',
        'verbose': False
    }

    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)
    preds = model.predict(X_test)
    f1 = f1_score(y_test, preds)
    return f1

# Оптимизация гиперпараметров
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-02-08 16:23:15,869] A new study created in memory with name: no-name-4fef7c9a-4040-4154-935f-f73b1e269509
[I 2025-02-08 16:23:16,015] Trial 0 finished with value: 0.8310679611650486 and parameters: {'iterations': 292, 'learning_rate': 0.21219992247461122, 'depth': 7, 'l2_leaf_reg': 1.1247756576421302, 'border_count': 176, 'random_strength': 9.091165388733742, 'bagging_temperature': 0.8216278974012712}. Best is trial 0 with value: 0.8310679611650486.
[I 2025-02-08 16:23:16,274] Trial 1 finished with value: 0.8400770712909441 and parameters: {'iterations': 520, 'learning_rate': 0.05687263190758436, 'depth': 8, 'l2_leaf_reg': 4.172455035776009, 'border_count': 153, 'random_strength': 2.263037493627998, 'bagging_temperature': 0.6781283454623216}. Best is trial 1 with value: 0.8400770712909441.
[I 2025-02-08 16:23:16,366] Trial 2 finished with value: 0.8477842003853564 and parameters: {'iterations': 739, 'learning_rate': 0.17795333438406982, 'depth': 6, 'l2_leaf_reg': 7.129970393232

In [46]:
# Обучение модели с лучшими параметрами и оценка
best_params = study.best_params
best_params['auto_class_weights'] = 'Balanced'
best_model = CatBoostClassifier(**best_params, verbose=False)
best_model.fit(X_train, y_train)

# Оценка модели
test_preds = best_model.predict(X_test)
print(classification_report(y_test, test_preds))
print(f"ROC-AUC: {roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])}")

              precision    recall  f1-score   support

           0       0.58      0.56      0.57        68
           1       0.89      0.90      0.90       275

    accuracy                           0.83       343
   macro avg       0.74      0.73      0.73       343
weighted avg       0.83      0.83      0.83       343

ROC-AUC: 0.8081550802139038


In [50]:
# Важность признаков
feature_importances = best_model.get_feature_importance()
for name, importance in zip(X.columns, feature_importances):
    print(f"{name}: {importance:.2f}%")


Важность признаков:
       feature  importance
0         NDVI   36.158279
1          LST   34.879567
2  BURNED_AREA   28.962154


Значение Roc Auc - 0.80+ означает. Модель хорошо разделяет классы, но не идеально.

Выводы
1. Класс 1 предсказывается значительно лучше, чем 0.
2. Модель в целом работает хорошо (accuracy = 83%), но баланс классов можно улучшить.

Для класса 0 (нет пожара) низкие precision и recall, возможно, стоит рассмотреть:
- Использование взвешивания классов (class_weight='balanced').
- Добавление больше данных для класса 0.
- Улучшение модели, например, за счет оптимизации порога классификации.